In [1]:
from dotenv import load_dotenv
import os
import pymongo
import os
import pandas as pd
from pymongo import MongoClient

In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [3]:
# Retrieve the MongoDB connection URI from the environment variables
dbURI = os.getenv("MONGODB_URI")

In [4]:
# Connect to MongoDB using the connection URI
client = pymongo.MongoClient(dbURI)

In [5]:
# List all the databases in the MongoDB server
databases = client.list_database_names()

In [6]:
# Print the list of database names
print("Databases in the MongoDB server:")
for db_name in databases:
    print(db_name)

Databases in the MongoDB server:
Mongo
Project
pymongo
pymongodb
sample_airbnb
sample_analytics
sample_geospatial
sample_guides
sample_mflix
sample_restaurants
sample_supplies
sample_training
sample_weatherdata
admin
local


In [7]:
# Access the "Project" database (it's not physically created until it has content)
db = client['Project']

In [21]:
# Explicitly create the "capacity" collection
db.create_collection('capacity')
collection = db['capacity']

In [22]:
# Explicitly create the "generation" collection
db.create_collection('generation')
collection1 = db['generation']

In [12]:
# List all collections (they won't appear in the list until they contain at least one document)
print("Collections in the 'Project' database after attempted creation:")
for collection_name in db.list_collection_names():
    print(collection_name)

Collections in the 'Project' database after attempted creation:
capacity_by_sector
generation
capacity


In [23]:
# Path folder CSV capacity files
folder_path = 'capacity_csv'

In [18]:
# Drop all documents in the collection
# collection2.delete_many({})

In [42]:
# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file
        data = pd.read_csv(file_path, index_col=0)

        # Convert index to string if it's not
        data.index = data.index.map(str)

        # Fix the year columns by converting float to int, then to string if they are numbers
        data.columns = [str(int(float(col))) if col.isdigit() or col.replace('.', '', 1).isdigit() else col for col in data.columns]

        # Extract the province name from the file name
        province_name = filename.replace('Capacity in ', '').split('.')[0]

        # Initialize the document with the province name as _id
        document = {"_id": province_name}

        # Add each energy type with years as subdocuments
        for energy_type, row in data.iterrows():
            annual_data = {year: value for year, value in row.to_dict().items()}
            document[energy_type] = annual_data

        # Insert the data into MongoDB, replacing the existing document with the same _id
        collection.replace_one({"_id": province_name}, document, upsert=True)
        print(f"Data inserted for {province_name}")

Data inserted for Alberta
Data inserted for British Columbia
Data inserted for Canada
Data inserted for Manitoba
Data inserted for New Brunswick
Data inserted for Newfoundland and Labrador
Data inserted for Northwest Territories
Data inserted for Nova Scotia
Data inserted for Nunavut
Data inserted for Ontario
Data inserted for Prince Edward Island
Data inserted for Quebec
Data inserted for Saskatchewan
Data inserted for Yukon


In [43]:
# Path folder CSV capacity files
folder_path1 = 'generation_csv'

In [44]:
# Process each file in the folder
for filename in os.listdir(folder_path1):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path1, filename)

        # Read the CSV file
        data = pd.read_csv(file_path, index_col=0)

        # Convert index to string if it's not
        data.index = data.index.map(str)

        # Fix the year columns by converting float to int, then to string if they are numbers
        data.columns = [str(int(float(col))) if col.isdigit() or col.replace('.', '', 1).isdigit() else col for col in data.columns]

        # Extract the province name from the file name
        province_name = filename.replace('Generation in ', '').split('.')[0]

        # Initialize the document with the province name as _id
        document = {"_id": province_name}

        # Add each energy type with years as subdocuments
        for energy_type, row in data.iterrows():
            annual_data = {year: value for year, value in row.to_dict().items()}
            document[energy_type] = annual_data

        # Insert the data into MongoDB, replacing the existing document with the same _id
        collection1.replace_one({"_id": province_name}, document, upsert=True)
        print(f"Data inserted for {province_name}")

Data inserted for Alberta
Data inserted for British Columbia
Data inserted for Canada
Data inserted for Manitoba
Data inserted for New Brunswick
Data inserted for Newfoundland and Labrador
Data inserted for Northwest Territories
Data inserted for Nova Scotia
Data inserted for Nunavut
Data inserted for Ontario
Data inserted for Prince Edward Island
Data inserted for Quebec
Data inserted for Saskatchewan
Data inserted for Yukon


In [11]:
# Explicitly create the "capacity_by_sector" collection
db.create_collection('capacity_by_sector')
collection2 = db['capacity_by_sector']

In [13]:
# Path folder CSV capacity files
folder_path2 = 'private_public_capacity'

In [19]:
# Process each file in the folder
for filename in os.listdir(folder_path2):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path2, filename)

        # Read the CSV file
        data = pd.read_csv(file_path, index_col=0)

        # Convert index to string
        data.index = data.index.map(str)

        # Fix the year columns by converting float to int, then to string if they are numbers
        #data.columns = [str(int(float(col))) if col.isdigit() or col.replace('.', '', 1).isdigit() else col for col in data.columns]

        # Extract the sector name from the file name
        sector_name = filename.replace('capacity in Electricity producer, ', '').split('.')[0]

        # Initialize the document with the province name as _id
        document = {"_id": sector_name}

         # Iterate over each row in the DataFrame and construct the subdocuments
        for index, row in data.iterrows():
            year = str(row['REF_DATE'])
            document[year] = {
                "UOM": row['UOM'],
                "VALUE": row['VALUE'],
                "Grow_percentage": row['Porcentaje_Aumento']
            }

        # Insert the data into MongoDB, replacing the existing document with the same _id
        collection2.replace_one({"_id": sector_name}, document, upsert=True)
        print(f"Data inserted for {sector_name}")

Data inserted for industries
Data inserted for private electric utilities
Data inserted for public electric utilities
Data inserted for capacity in Total all classes of electricity producer
